# Problem set 9: Statistics, feature selection, and feature importance (solution)

## Summary

Examine the differences between British and American fiction in the class-curated literary corpus. Apply statistical measures and calculate feature importance in a simple classifier.

## Details

You will work with a corpus of 131 volumes of fiction by British and American authors. These volumes are taken from the class corpus, so you'll need to download a [copy of the texts](https://drive.google.com/drive/folders/1lbeZiBAVCzjCWojCK8mfmELa-Q8FMNUm?usp=sharing) and save them somewhere on your machine.

You have three tasks for this problem set, all of which depend on comparing British-authored to American-authored books:

1. Calculate the mean frequency per 100,000 words, as well as the upper and lower bounds of a 95% confidence interval, for the terms `['color', 'honor', 'center', 'fish', 'person']`
    1. Perform this calculation analyticaly, that is, using the observed sample means and standard deviations. I suggest using the `tconfint_mean()` method from the `DescrStatsW()` function provided by the `statsmodels` library. See lecture notes for an example of working code.
    1. Calculate the same quantities via bootstrap, using 1,000 or more iterations.
    1. In both cases, print your results in a tabular format, like so:

```
Confidence intervals for: gb
     term	    low	    mean	    high
   color	  x.xxxx	  x.xxxx	  x.xxxx
   [and so on ...]
```

2. Perform a *t*-test to compare the mean frequency of each of these terms between British and American texts. Report the test statistic and *p*-value for each comparison. Note which means are significantly different at the *p*<0.05 level.
3. Perform a logistic regression classification of each volume as British or American. 
    1. Your final features should be the 25 most informative (as measured by the mutual information criterion) token unigrams.
    1. Report your 10-fold cross-validated F1 score before and after restricting your input features to the 25 most-informative token types.
    1. Calculate the *importance* of the 25 top features for classification as measured by permutation importance.
    
* See code stubs below for step-by-step guidance. 
* Consult, too, the lecture notes on explainability and on statistics.
* You'll likely also need to consult the scikit-learn documentation along the way.

## Imports and setup

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np

metadata_file = 'amer_brit.csv'
corpus_dir = os.path.join('..', '..', 'data', 'classcorpus')
terms = ['color', 'honor', 'center', 'fish', 'person']

## Read metadata (5 points)

Read the cleaned, minimal corpus metadata from disk (note the variable `metadata_file` defined in the previous cell). I'd suggest using Pandas, but you're welcome to use whatever method you prefer.

Note that the format of the metadata file is:

```
filename,country,wordcount
```

In [3]:
# Read the corpus metadata
corpus = pd.read_csv(metadata_file)

# Print the metadata for one volume
corpus.head(1)

,filename,country,wc
0,Little_Women_Alcott.txt,us,185902


Note that there's a duplicate volume:

In [4]:
corpus.loc[corpus.filename.str.lower()=='plague_ship.txt']

,filename,country,wc
87,Plague_Ship.txt,us,60020
88,plague_ship.txt,us,60020


We discovered this after the problem set went live, so we'll leave it alone. But in the future, we'd want to remove one of the copies. These kinds of problems are typical of real-world corpora. To remove the duplicate, we could either exclude its index position or (more generically) drop any rows with identical properties *other* than filename:

```
corpus = corpus.drop_duplicates(subset=corpus.columns[1:])
```

## Count words and normalize (5 points)

* Count the target words (indicated in the problem statement) in each volume. 
* Then, normalize the count of each word type per 100,000 words in each volume*  I'd suggest using a `CountVectorizer` object, but again, you may approach this task however you like. 
* Make sure you lowercase the input tokens.
* Use the word counts supplied in the metadata file for length normalization.

In [5]:
# Count and normalize the all terms in each volume as indicated
from sklearn.feature_extraction.text import CountVectorizer

count_vec = CountVectorizer(
    input='filename',
    encoding='utf-8',
    strip_accents='unicode',
    lowercase=True
)

file_list = corpus.filename.apply(lambda x: os.path.join(corpus_dir, x))
count_data = count_vec.fit_transform(file_list)

In [6]:
# An alternative, limiting initial counts to just the target terms
count_vec = CountVectorizer(
    input='filename',
    encoding='utf-8',
    strip_accents='unicode',
    lowercase=True,
    vocabulary=terms # <- note this option
)

file_list = corpus.filename.apply(lambda x: os.path.join(corpus_dir, x))
count_data = count_vec.fit_transform(file_list)

In [7]:
# Print the normalized term frequencies you just calculated for any three documents
for term in terms: # select just the target terms from the full matrix
    corpus[term] = count_data[:,count_vec.vocabulary_[term]].T.toarray()[0]
    corpus[term] = corpus[term]/corpus['wc']*100000
corpus.head(3)

,filename,country,wc,color,honor,center,fish,person
0,Little_Women_Alcott.txt,us,185902,11.296274,15.599617,0.000000,3.765425,12.372110
1,The_Great_God_Pan.txt,gb,21639,0.000000,0.000000,0.000000,9.242571,13.863857
2,The_Lost_Kafoozalum.txt,gb,22371,0.000000,0.000000,4.470073,26.820437,8.940146


## Calculate analytic means and 95% confidence intervals (15 points)

For each of the five indicated terms, calculate and display the mean and 95% confidence interval within each national group. See instructions above for output format.

In this part of the problem, calculate your means and CIs analytically, using the observed statistics of each sample, rather than by bootstrapping.

In [8]:
import statsmodels.stats.api as sms
g = corpus.groupby('country')
print("Analytic confidence intervals")
for c, group in g:
    print("\nConfidence intervals for:", c)
    print("     term\t     low\t    mean\t    high")
    for term in terms:
        mean = group[term].mean()
        low, high = sms.DescrStatsW(group[term]).tconfint_mean()
        print(f'   {term:<5}\t{low:8.4f}\t{mean:8.4f}\t{high:8.4f}')

Analytic confidence intervals

Confidence intervals for: gb
     term	     low	    mean	    high
   color	  0.0573	  0.6308	  1.2042
   honor	 -0.2501	  0.4263	  1.1027
   center	 -0.0075	  0.5890	  1.1855
   fish 	  1.7177	  4.2106	  6.7035
   person	 22.3781	 27.0922	 31.8064

Confidence intervals for: us
     term	     low	    mean	    high
   color	  8.6397	 12.8857	 17.1316
   honor	  4.6014	  6.8694	  9.1373
   center	  4.0506	  6.4329	  8.8153
   fish 	  3.7917	  8.4189	 13.0460
   person	 16.1344	 20.5517	 24.9690


## Calculate bootstrapped means and 95% confidence intervals (15 points)

* Calculate the same quantities as above, but this time by bootrap resampling of your data. 
* Use a minimum of 1,000 trials for each case. 
* Format your results as in the previous question.

In [9]:
# Bootstrap calculations
trials = 1000
print("Bootstrap confidence intervals")
for c, group in g:
    print("\nConfidence intervals for:", c)
    print("     term\t     low\t    mean\t    high")
    for term in terms:
        bootstrapped_means = []
        k = group[term].count() # Number of objects per sample
        for i in range(trials):
            sample = group[term].sample(n=k, replace=True)
            bootstrapped_means.append(np.mean(sample))
        result = sorted(bootstrapped_means)
        mean = result[int(trials/2)]
        low = result[int(trials*0.025)]  # 2.5%
        high = result[int(trials*0.975)] # 97.5%
        print(f'   {term:<5}\t{low:8.4f}\t{mean:8.4f}\t{high:8.4f}')

Bootstrap confidence intervals

Confidence intervals for: gb
     term	     low	    mean	    high
   color	  0.1321	  0.5987	  1.2706
   honor	  0.0264	  0.3999	  1.1557
   center	  0.1310	  0.5706	  1.2830
   fish 	  2.1171	  4.1516	  7.0727
   person	 22.5262	 27.1448	 31.6609

Confidence intervals for: us
     term	     low	    mean	    high
   color	  8.9852	 12.7413	 17.4853
   honor	  4.8130	  6.8344	  9.2796
   center	  4.3453	  6.3913	  8.9628
   fish 	  4.4221	  8.3615	 13.5650
   person	 16.5477	 20.5420	 25.4445


## *t*-tests (20 points)

* Perform a *t*-test comparing the mean frequency of each of the indicated terms in the British and American subsets of the corpus.
    * You will perform 5 total tests, comparing, for example, the mean frequency of `color` in British texts to the mean frequency of `color` in American texts. Do not cross-compare words (that is, don't compare the frequency of `color` to that of `honor`, etc.).
* Display the test statistic and *p*-value for each comparison. 
    * Format your output for easy readability (do not just print the raw `ttest_ind` object).
* Note which differences are significant at the *p*<0.05 level. 

In [10]:
from scipy.stats import ttest_ind
print("term\tstatistic\t  p-value")
for term in terms:
    stat, p = ttest_ind(
        corpus.loc[corpus.country=='us'][term], 
        corpus.loc[corpus.country=='gb'][term],
        equal_var=False
    )
    print(f"{term}\t{stat:9.4}\t{p:9.4}")

term	statistic	  p-value
color	    5.711	2.662e-07
honor	    5.436	6.096e-07
center	    4.751	9.649e-06
fish	    1.599	    0.113
person	   -2.022	  0.04523


* **Significant:** color, honor, center, person (*marginal*)
* **Not sigificant:** fish

## Feature selection (25 points)

* Vectorize the corpus as indicated below (freebie)
* Standard-scale the resulting feature matrix
* Produce a one-dimensional label vector, y, indicating the national origin of each volume in the corpus
    * Use `1` to indicate American, `0` for British
* Calculate the 10-fold cross-validated classification accuracy and F1 score using a logistic regression classifier on the full input matrix
* From the full matrix, select the 25 most-informative features
    * Use sklearn's `SelectKBest` function with the  `mutual_info_classif` scoring function to produce a feature matrix that contains just these 25 most-informative features
    * Print a list of the names (token labels; for example, 'color') of these 25 features

In [11]:
# Vectorize (freebie)
from sklearn.feature_extraction.text import TfidfVectorizer

def pre_proc(x):
    '''
    Takes a unicode string.
    Lowercases, strips accents, and removes some escapes.
    Returns a standardized version of the string.
    '''
    import unicodedata
    return unicodedata.normalize('NFKD', x.replace("_", " ").lower().strip())

# Set up vectorizer
vectorizer = TfidfVectorizer(
    input='filename',
    encoding='utf-8',
    preprocessor=pre_proc,
    min_df=11, # Note this
    max_df=0.8, # This, too
    binary=False,
    norm='l2',
    max_features=5000,
    use_idf=True # And this
)

# Perform vectorization
X = vectorizer.fit_transform(file_list) # <-- MODIFY TO USE THE LIST OF FILES ON YOUR MACHINE

# Get the dimensions of the doc-term matrix
print("Matrix shape:", X.shape)

Matrix shape: (131, 5000)


In [12]:
# Standard-scale your feature matrix
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X.todense())

# Print the mean of your scaled features.
# Should be very close to zero.
print("Mean scaled value:", np.mean(X))

Mean scaled value: -6.942707647121589e-18


In [13]:
# Produce a one-dimensional vector of true labels for classification
# 1='us', 0='gb'
from sklearn.preprocessing import LabelBinarizer
y = LabelBinarizer().fit_transform(corpus.country.to_numpy().reshape(-1, 1)).ravel()

# Using your label vector, display the number of US texts in the corpus
print("Number of US texts:", np.sum(y))

Number of US texts: 67


In [15]:
# Cross-validate the logistic regression classifier on full input data
# Consult PS 6 for leads
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

# Classifers to test
classifiers = {
    'Logit':LogisticRegression()
}

scores = {} # Store cross-validation results in a dictionary
for classifier in classifiers: 
    scores[classifier] = cross_validate( # perform cross-validation
        classifiers[classifier], # classifier object
        X, # feature matrix
        y, # gold labels
        cv=10, #number of folds
        scoring=['accuracy', 'f1'] # scoring methods
    )

In [16]:
# Examine the performance of our classifier
# Freebie function to summarize and display classifier scores
def compare_scores(scores_dict, color=True):
    '''
    Takes a dictionary of cross_validate scores.
    Returns a color-coded Pandas dataframe that summarizes those scores.
    '''
    import pandas as pd
    df = pd.DataFrame(scores_dict).T.applymap(np.mean)
    if color:
        df = df.style.background_gradient(cmap='RdYlGn')
    return df

In [17]:
# Compare cross-validation scores
# Note that colorization of the `time` columns is counterintuitive
compare_scores(scores, color=False)

,fit_time,score_time,test_accuracy,test_f1
Logit,0.06704,0.001427,0.847802,0.843337


In [18]:
# Select the 25 most-informative features as specified above 
#  and produce a new feature matrix containing only those features
from sklearn.feature_selection import SelectKBest, mutual_info_classif
selector = SelectKBest(mutual_info_classif, k=25)
X_selected = selector.fit_transform(X, y)

# Print the shape of your new feature matrix
print("Shape of the new feature matrix:", X_selected.shape)

Shape of the new feature matrix: (131, 25)


In [16]:
# Get the names of the features retained in the new feature matrix
# Store these feature names in a list, then print the list

# Hint: use a combination of your original vectorizer's `.get_feature_names()` method 
#  and the `SelectKBest` object's `.get_support()` method
feature_names = [x for i, x in enumerate(vectorizer.get_feature_names()) if selector.get_support()[i]]
display(feature_names)

['afterwards',
 'british',
 'color',
 'colored',
 'coloured',
 'endeavoured',
 'endeavouring',
 'events',
 'extraordinary',
 'favor',
 'favour',
 'grey',
 'honour',
 'honourable',
 'horizon',
 'humour',
 'inn',
 'london',
 'neighbourhood',
 'realise',
 'remainder',
 'sore',
 'toward',
 'towards',
 'traveling']

In [17]:
# Calculate and display the 10-fold cross-validated accuracy and F1 of the
#  logistic regression using the new, smaller feature matrix
scores = {} # Store cross-validation results in a dictionary
for classifier in classifiers: 
    scores[classifier] = cross_validate( # perform cross-validation
        classifiers[classifier], # classifier object
        X_selected, # feature matrix
        y, # gold labels
        cv=10, #number of folds
        scoring=['accuracy', 'f1'] # scoring methods
    )
compare_scores(scores, color=False)

,fit_time,score_time,test_accuracy,test_f1
Logit,0.008607,0.001269,0.878571,0.878473


## Identify the 5 most important features (15 points)

* Split the new matrix of most-informative features into train (75%) and test (25%) sets (use sklearn's `train_test_split`)
* Train a default logistic regression classifier on the training set
    * Print the trained model's score on the test set (use the trained classifier's `.score()` method)
* Use sklearn's `permutation_importance` function to calculate the importance of each input feature
* Print the feature importances from most to least important using the supplied function

In [22]:
# Split the selected feature matrix into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_selected, y)
model = LogisticRegression().fit(X_train, y_train)

print("Test score:", model.score(X_val, y_val))

Test score: 0.9393939393939394


In [23]:
# Calculate feature importance via permutation
from sklearn.inspection import permutation_importance
r = permutation_importance(model, X_val, y_val, n_repeats=500)

In [24]:
def print_importances(importance_object, feature_names):
    '''
    Takes a trained permutation_importance object and a list of feature names.
    Prints an ordered list of features by descending importance.
    '''
    for i in importance_object.importances_mean.argsort()[::-1]:

        print(f"{feature_names[i]:<8}"
            f"\t{importance_object.importances_mean[i]:.3f}"
            f" +/- {importance_object.importances_std[i]:.3f}")

In [25]:
# Print ranked list of features by permutation importance
print_importances(r, feature_names)

british 	0.054 +/- 0.030
toward  	0.050 +/- 0.032
afterwards	0.041 +/- 0.034
color   	0.038 +/- 0.035
honour  	0.016 +/- 0.029
endeavoured	0.015 +/- 0.018
inn     	0.013 +/- 0.018
favor   	0.013 +/- 0.023
horizon 	0.011 +/- 0.017
remainder	0.006 +/- 0.013
extraordinary	0.003 +/- 0.008
endeavouring	0.001 +/- 0.005
grey    	0.001 +/- 0.005
sore    	0.000 +/- 0.000
events  	0.000 +/- 0.000
realise 	0.000 +/- 0.000
neighbourhood	0.000 +/- 0.000
london  	0.000 +/- 0.000
traveling	0.000 +/- 0.000
towards 	-0.001 +/- 0.018
favour  	-0.002 +/- 0.013
colored 	-0.002 +/- 0.013
coloured	-0.002 +/- 0.013
humour  	-0.002 +/- 0.008
honourable	-0.017 +/- 0.015
